This notebook aims to compute the descriptor for combination of two "center" atoms

# Create a test system

In [ ]:
import ase

atoms = ase.Atoms("SSNO", positions=[[0, 0, 0], [0, 0, 0.1], [0, 0, 1], [0, 0, 2]])
frames = [atoms]

# Common Hyperparameters

In [ ]:
r_cut = 2.5
n_max = 12
l_max = 12
sigma = 0.3

In [ ]:
# this is for one frame only for now...

list_S = [1, 2]  # list of all indices we label as "start" atom
list_M = [2, 3]  # list of all indices we label as "middle" atom
list_E = [3, 1]  # list of all indices we label as "end" atom

assert len(list_S) == len(list_M)
assert len(list_S) == len(list_E)

# `dscribe` descriptor

In [ ]:
from dscribe.descriptors import SOAP

soaper = SOAP(
    r_cut=r_cut,
    n_max=n_max,
    l_max=l_max,
    sigma=sigma,
    sparse=False,
    species=["S", "O", "N"],
)

In [ ]:
soap_water = soaper.create(frames[0], centers=list_S)

# pair descriptor

The code for the descriptor calculations is extracted from 

https://github.com/curiosity54/mlelec

In [ ]:
from utils.acdc import pair_features

In [ ]:
hypers = {
    "cutoff": r_cut,
    "max_radial": n_max,
    "max_angular": l_max,
    "atomic_gaussian_width": sigma,
    "center_atom_weight": 1,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.1}},
}

In [ ]:
return_rho0ij = False
both_centers = False
all_pairs = False

rhoij = pair_features(
    frames=[atoms],
    hypers=hypers,
    cg=None,
    order_nu=1,
    all_pairs=all_pairs,
    both_centers=both_centers,
)

In [ ]:
# list of center pairs
# frame_index, i, j
list_ij = np.array([[0, s, e] for s,e in zip(list_S, list_E)])


In [ ]:
# convert list of indices to list of species
species_ij = []
for ifr, i, j in list_ij:
    atomic_species = frames[ifr].numbers
    species_i = atomic_species[i]
    species_j = atomic_species[j]

    species_ij.append((species_i, species_j))

In [ ]:
species_ij
unique_species_ij, inverse = np.unique(species_ij, return_inverse=True, axis=0)

In [ ]:
pair_soap_features = []

for inverse_index, (species_i, species_j) in enumerate(unique_species_ij):
    block = rhoij.block(spherical_harmonics_l=0, inversion_sigma=1, species_center=species_i, species_neighbor=species_j)

    values = block.values
    sample_values = block.samples.values

    mask = inverse == inverse_index
    selected_samples = list_ij[mask]

    value_indices = np.array([np.where(np.all(sample_values == s, axis=1)) for s in selected_samples])

    values_selected = values[value_indices]

    pair_soap_features.append(values_selected.numpy().flatten())

Number of features rows is the same between describe and pair features...

In [ ]:
len(pair_soap_features)

In [ ]:
len(soap_water)